# Modeling the heads in the Baltics
*M. Jemeljanova, R.A. Collenteur (November 2022)*

This notebook is part of the manuscript by Jemeljanova et al. titled "*Modeling hydraulic heads with impulse response functions in different environmental settings*" submitted to Journal of Hydrology: Regional Studies.

## 0. Import python packages

In [ ]:
import os
import gc
import numpy as np
import pandas as pd
import numba as numba

import matplotlib.pyplot as plt

import pastas as ps
import pyet

from tqdm import tqdm

ps.set_log_level("ERROR")
ps.show_versions(numba=True)

## 1. Load data

In [ ]:
os.getcwd()
os.chdir('../') 

heads = pd.read_csv(os.path.join(os.getcwd(), 'input', 'hydraulic_head.csv'), 
                    parse_dates=['date'], index_col='date')
precipitation = pd.read_csv(os.path.join(os.getcwd(), 'input', 'precipitation.csv'), 
                            parse_dates=['time'], index_col='time')
evaporation = pd.read_csv(os.path.join(os.getcwd(), 'input', 'evaporation.csv'), 
                          parse_dates=['time'], index_col='time')
temperature = pd.read_csv(os.path.join(os.getcwd(), 'input', 'temperature_mean.csv'), 
                          parse_dates=['time'], index_col='time')

## 2. Define helper functions

In [ ]:
def model_creation(head, prec, evap, tmean):
    """Method to create and calibrate the four different model structures
    
    Parameters
    ----------
    head: pandas.Series
        Pandas time series with the heads.
    prec: pandas.Series
        Pandas time series with the precipitation in mm/d.
    evap: pandas.Series
        Pandas time series with the potential evaporation in mm/d.
    tmean: pandas.Series
        Pandas time series with the average daily temperature in degree
        Celcius.
        
    """
    # LINEAR + GAMMA
    
    model_lg = ps.Model(head, name="LG")
    recharge_Lin_G = ps.RechargeModel(prec, evap, rfunc=ps.Gamma, recharge= ps.rch.Linear(), name = "rch")
    model_lg.add_stressmodel(recharge_Lin_G) # adds recharge for parameter calibration
    # first calibrate model without noise to bet better initial parameters
    model_lg.solve(noise=False, tmin=tmin_cal, tmax=tmax_cal, report=False)
    # calibrate with noise
    model_lg.solve(noise=True, initial=False, tmin=tmin_cal ,tmax=tmax_cal, report=False) 

    # LINEAR + FOUR PARAMETER
    
    model_l4 = ps.Model(head, name="L4")
    recharge_Lin_4 = ps.RechargeModel(prec, evap, rfunc=ps.FourParam, recharge= ps.rch.Linear(), name="rch")
    model_l4.add_stressmodel(recharge_Lin_4)
    model_l4.solve(noise=False, tmin=tmin_cal, tmax=tmax_cal, report=False)
    model_l4.solve(noise=True, initial=False, tmin=tmin_cal ,tmax=tmax_cal, report=False) 

    # NON-lINEAR (Flex) + GAMMA
    
    model_nlg = ps.Model(head, name="NLG")
    recharge_Flex_G = ps.RechargeModel(prec, evap, rfunc=ps.Gamma, recharge= ps.rch.FlexModel(), name="rch")
    model_nlg.add_stressmodel(recharge_Flex_G)
    model_nlg.set_parameter("rch_kv", vary=True, pmin=0.25)
    model_nlg.solve(noise=False, tmin=tmin_cal, tmax=tmax_cal, report=False)
    model_nlg.set_parameter("rch_srmax", vary=False)
    model_nlg.solve(noise=True, initial=False, tmin=tmin_cal ,tmax=tmax_cal, report=False)
    
    # NON-LINEAR (Flex) SNOW
    
    model_s = ps.Model(head, name="NLS")
    recharge_Flex_s = ps.RechargeModel(prec, evap, rfunc=ps.Gamma, recharge=ps.rch.FlexModel(snow=True), 
                                       name="rch", temp=tmean)
    model_s.add_stressmodel(recharge_Flex_s)
    model_s.set_parameter("rch_kv", vary=True, pmin=0.25)
    model_s.solve(noise=False, tmin=tmin_cal, tmax=tmax_cal, report=False)
    model_s.set_parameter("rch_srmax", vary=False)
    model_s.solve(noise=True, initial=False, tmin=tmin_cal ,tmax=tmax_cal, report=False)    
    
    return model_lg, model_l4, model_nlg, model_s

In [ ]:
# Create metrics DataFrame
stats = ['RMSE_c', 'NSE_c', 'KGE_c', 'RMSE_v', 'NSE_v', 'KGE_v']
mi = pd.MultiIndex.from_product([heads.columns[1:].unique(), ["LG", "L4", "NLG", "NLS"]])
all_metrics = pd.DataFrame(index=mi, columns=stats)

def calculating_metrics(well, models):  
    """Method to compute the metrics for all models
    
    Parameter
    ---------
    well: str
        String with the name of the well
    models: lost of Pastas.Models
        List with the Pastas models.
    
    """
    for model in models:

        # Metrics in the calibration period
        rmse_c = model.stats.rmse(tmin = tmin_cal, tmax = tmax_cal)
        nse_c = model.stats.nse(tmin = tmin_cal, tmax = tmax_cal)
        kge_c = model.stats.kge_2012(tmin = tmin_cal, tmax = tmax_cal)

        # Metrics in the valivation period
        rmse_v = model.stats.rmse(tmin = tmin_val, tmax = tmax_val)
        nse_v = model.stats.nse(tmin = tmin_val, tmax = tmax_val)
        kge_v = model.stats.kge_2012(tmin = tmin_val, tmax = tmax_val)

        all_metrics.loc[(well, model.name), stats] = [rmse_c, nse_c, kge_c, 
                                                      rmse_v, nse_v, kge_v]

In [ ]:
# Create parameters DataFrame
names = ['rch_A', 'rch_n', 'rch_a']
mi = pd.MultiIndex.from_product([heads.columns[1:].unique(), ["LG", "L4", "NLG", "NLS"]])
params = pd.DataFrame(index=mi, columns=names)

def store_model_parameters(well, models):
    """Method to store the model parameters for all models.
    
    Parameter
    ---------
    well: str
        String with the name of the well
    models: lost of Pastas.Models
        List with the Pastas models.
    
    """
    for model in models:
        params.loc[(well, model.name), names] = model.parameters.loc[names, "optimal"]

In [ ]:
def saving_models(well, models):  
    """Method to save a pas-file all models.

    Parameter
    ---------
    well: str
        String with the name of the well
    models: lost of Pastas.Models
        List with the Pastas models.
    
    """
    for model in models:
        filename = str(well)[:2]+ str(well)[3:]+"_"+ model.name
        model.to_file((os.path.join(os.getcwd(), 'output', 'models', filename + ".pas")))

In [ ]:
def plotting_and_saving_comparison_figures(well, models):
    """Method to plot all models and save the figure.

    Parameter
    ---------
    well: str
        String with the name of the well
    models: lost of Pastas.Models
        List with the Pastas models.
    
    """
    ps.plots.compare(models, figsize=(20,10))
    plt.savefig(os.path.join(os.getcwd(), 'output', 'figures', well + ".png"))
    plt.close()

## 3. Run models

In [ ]:
for well in tqdm(heads.columns[1:].unique()):
    # select the data for the specific observation well
    head = heads[well].dropna()
    prec = precipitation[well].dropna()
    tmean = temperature[well].dropna()
    evap = evaporation[well].dropna().asfreq("D").interpolate() # interpolate a few entries
    
    # determine the calibration and validation dates from the meteodata     
    global tmax_cal, tmin_cal, tmax_val, tmin_val
    
    tmax_val = prec.index[-1]
    tmax_cal = tmax_val - pd.DateOffset(years = 5)
    tmin_val = tmax_cal + pd.DateOffset(days = 1)
    tmin_cal = tmin_val - pd.DateOffset(years = 10)
    
    # Make string for Pastas
    tmin_val = tmin_val.strftime("%Y-%m-%d")
    tmin_cal = tmin_cal.strftime("%Y-%m-%d")
    tmax_cal = tmax_cal.strftime("%Y-%m-%d")
    tmax_val = tmax_val.strftime("%Y-%m-%d")
    
    # Create and calibrate the models
    models = model_creation(head, prec, evap, tmean)
    
    # Extract information and store models
    calculating_metrics(well, models)
    store_model_parameters(well, models)
    saving_models(well, models)
    plotting_and_saving_comparison_figures(well, models)
    
    # Free up memory right away, no need to store
    gc.collect()

## 4. Export fit metrics

In [ ]:
params.to_csv(os.path.join(os.getcwd(), 'output', 'params.csv'))
params.dropna().head()

## 5. Export models parameters

In [ ]:
all_metrics.to_csv(os.path.join(os.getcwd(), 'output', 'metrics.csv'))
all_metrics.dropna().head()